<a href="https://colab.research.google.com/github/ghaitsafiya/STKI/blob/main/boolean_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Eksperimen Information Retrieval: Boolean Model
Eksperimen ini bertujuan untuk memahami cara kerja Boolean Model pada sistem temu kembali informasi (Information Retrieval) menggunakan kumpulan teks dari beberapa dokumen.

Boolean Model memungkinkan pengguna mencari dokumen dengan kombinasi logika seperti AND, OR, dan NOT.

In [2]:
from zipfile import ZipFile

with ZipFile("minggu3.zip", 'r') as zip_ref:
    zip_ref.extractall("data")

In [3]:
import os
import re
from collections import defaultdict

## Persiapan Data Set
Pada tahap ini, dokumen teks dikumpulkan aga bisa diproses dalam sistem pencarian.

In [4]:
DATA_DIR = "data"  # folder berisi file teks
docs = {}

for fname in os.listdir(DATA_DIR):
    if fname.endswith(".txt"):
        with open(os.path.join(DATA_DIR, fname), encoding="utf-8") as f:
            docs[fname] = f.read()

## Preprocessing Teks


*  Mengubah huruf menjadi kecil (lowercase).
*   Menghapus tanda baca dan karakter selain huruf/angka
*   Memecah teks menjadi token (kata-kata).



In [5]:
def tokenize(text):
    text = text.lower()
    text = re.sub(r'[^a-z0-9\s]', ' ', text)
    return text.split()

## Membangun Inverted Index
Membuat struktur data dasar untuk pencarian cepat berdasarkan kata kunci.

In [6]:
inverted_index = defaultdict(set) #menggunakan set agar nama dokumen tidak duplikat

for fname, content in docs.items():
    terms = set(tokenize(content))
    for term in terms:
        inverted_index[term].add(fname)

## Implementasi Boolean Query
Adanya implementasi ini memungkinkan pengguna melakukan pencarian logika sederhana pada kumpulan dokumen.
Mendukung tiga operator logika utama:
*   AND: hanya dokumen yang mengandung semua kata.
*   OR: dokumen yang mengandung salah satu kata.
*   NOT: mengeluarkan dokumen yang mengandung kata tersebut.

Query seperti "modi AND obama" akan mencari dokumen yang berisi kedua kata.

In [7]:
def boolean_query(query):
    all_docs = set(docs.keys())
    tokens = query.lower().split()
    result = set()
    op = None

    for token in tokens:
        if token == "and":
            op = "and"
        elif token == "or":
            op = "or"
        elif token == "not":
            op = "not"
        else:
            term_docs = inverted_index.get(token, set())

            if op == "not":
                term_docs = all_docs - term_docs
                op = None

            if not result:
                result = term_docs
            elif op == "and":
                result = result & term_docs
            elif op == "or":
                result = result | term_docs
            op = None

    return result

## Uji Coba Query
Mengukur apakah sistem Boolean Model berhasil menemukan dokumen sesuai kombinasi kata kunci.

In [8]:
queries = [
    "narendra AND modi",
    "barack AND obama",
    "queen AND elizabeth",
    "badminton AND NOT baseball",
    "modi OR obama"
]

for q in queries:
    print(f"Query: {q}")
    print("Dokumen hasil:", boolean_query(q))
    print("-"*40)

Query: narendra AND modi
Dokumen hasil: set()
----------------------------------------
Query: barack AND obama
Dokumen hasil: set()
----------------------------------------
Query: queen AND elizabeth
Dokumen hasil: set()
----------------------------------------
Query: badminton AND NOT baseball
Dokumen hasil: set()
----------------------------------------
Query: modi OR obama
Dokumen hasil: set()
----------------------------------------
